# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [278]:
import pandas as pd
import random
import os
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [279]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'id': 0, 
        'username': 'janisourander@kamk.fi',
        'password': fake.md5(),
        'created': datetime(1290, 1, 15, 0, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
            'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'id': 0, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'id': 2, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'id': 3, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'id': len(devices), 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'id': len(details),
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [280]:
# Settings for path prefixing
container = 'S3'
prefix = 'staging'
tool = 'dms'
system = 'company_rds'

# Mapping
table_paths = {}

# Map of combinations to create: (db, table)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    # Generate rdbms-path and path to file.
    dt = f'{db}.{table}'
    dp = os.path.join(container, prefix, tool, system, db, table, 'LOAD00000001.parquet')
    
    # Append
    table_paths[dt] = dp

    # Create
    if not os.path.exists(os.path.dirname(dp)):
        os.makedirs(os.path.dirname(dp))

## Generate Datasets

### Customers

In [281]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,id,username,password,created,modified
0,0,janisourander@kamk.fi,135f027721f9840003efe860b37328e0,1290-01-15 00:00:00,1970-02-20 12:34:56
1,1,colleen45@yahoo.com,989e05bb848ad26f37d5d8cf53c60f8f,2020-01-15 13:37:00,2020-06-06 21:33:05
2,2,emily75@hotmail.com,f1d5f11cadcb3bee236690e25bdbe362,2020-01-15 14:12:43,2020-09-22 17:37:41
3,3,chrisfarrell@hotmail.com,e587e7cb20a566724a0541225b219016,2020-01-15 14:38:55,2020-08-03 11:51:18
4,4,swells@andrews-rowe.net,e66fcd21555908d4a4940e6462bba101,2020-01-15 14:42:01,2020-07-30 23:44:39
5,5,npearson@mathews.com,d67e01f1df84ca76d18335fdbf55fd10,2020-01-15 15:17:26,2020-08-30 20:21:12
6,6,grace05@hotmail.com,6f5915f11d91c2c3a3f78e92c4362bf8,2020-01-15 15:35:37,2020-12-20 01:37:41
7,7,cervantesjonathan@powers.com,901ae44b77ce9a6ef8e3e75c15977ad9,2020-01-15 16:11:25,2020-01-28 02:38:20
8,8,longjason@yahoo.com,6b51e478339dc0dcbcd4ab5f26886071,2020-01-15 16:23:21,2020-02-05 12:51:12
9,9,thompsonoscar@hotmail.com,8be6cb183765c577ff313404a6678f5f,2020-01-15 17:11:12,2020-07-25 07:34:49


### Device Models

In [282]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,id,release_date,name,color,description,created,modified
0,0,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,1,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,3,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [283]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_dm.to_parquet(table_paths['devices.devices'])

# Show
df_d

,id,customer_fk,model_fk,serial_number,created,modified
0,0,0,3,604-84-9532,1290-01-15 00:52:03,1290-01-15 01:36:17
1,1,0,1,454-97-8659,1290-01-15 00:52:03,1290-01-15 01:11:43
2,2,1,1,355-41-6610,2020-01-15 13:54:35,2020-01-15 14:37:45
3,3,2,1,217-73-9409,2020-01-15 14:30:15,2020-01-15 14:30:45
4,4,3,1,111-74-6295,2020-01-15 14:39:06,2020-01-15 14:57:43
5,5,3,2,219-89-1903,2020-01-15 14:39:06,2020-01-15 15:24:54
6,6,4,1,518-89-7343,2020-01-15 14:59:28,2020-01-15 15:29:49
7,7,5,1,369-88-4831,2020-01-15 15:29:24,2020-01-15 16:01:47
8,8,6,1,433-89-7867,2020-01-15 15:39:02,2020-01-15 16:25:52
9,9,6,0,788-92-7320,2020-01-15 15:39:02,2020-01-15 16:36:31


### Customer_details

In [284]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_dm.to_parquet(table_paths['customers.customer_details'])

df_cd

,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,0,0,1976-07-01,ps,82288 Jason Creek Suite 270,80711,South Stacey,TV,001-724-889-2036x5600,1290-01-15 00:00:00,1290-01-15 00:00:00
1,1,1,1908-03-17,wo,7905 Ashley Mission Suite 321,06823,South Sandra,IT,001-692-755-6205x1250,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2,2,1929-05-25,tr,48898 Hawkins Station Suite 445,56980,Samanthaberg,UZ,001-331-231-0856,2020-01-15 14:12:43,2020-01-15 14:12:43
3,3,3,1956-12-18,hak,33359 Andrews Inlet,31870,Stephenton,JM,128-170-5179,2020-01-15 14:38:55,2020-01-15 14:38:55
4,4,4,2011-09-03,ca,9784 Bowers Rapids Apt. 042,37536,Christopherhaven,BI,+1-604-642-2401x2057,2020-01-15 14:42:01,2020-01-15 14:42:01
5,5,5,2017-11-22,sat,88702 Mason Greens,87138,Lanemouth,DZ,6050624499,2020-01-15 15:17:26,2020-01-15 15:17:26
6,6,6,1973-03-28,mk,792 Roberson Lights,99520,North Karenfort,GE,865-370-2368,2020-01-15 15:35:37,2020-01-15 15:35:37
7,7,7,1907-04-09,zu,1168 Moss Gardens Suite 297,26180,Port Jack,CL,(421)039-7440x769,2020-01-15 16:11:25,2020-01-15 16:11:25
8,8,8,1921-08-25,as,126 Peterson Ville,18411,East Nicholasburgh,TT,645.576.0770x92061,2020-01-15 16:23:21,2020-01-15 16:23:21
9,9,9,1986-08-01,te,3409 Maria Cliff,42346,Donnahaven,AT,(965)533-3107x45494,2020-01-15 17:11:12,2020-01-15 17:11:12


## List resulting files

In [307]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()])

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f:<{max_w}} | {f_size:>2d} KB")

21-08-03 12:18:58 | S3\staging\dms\company_rds\customers\customers\LOAD00000001.parquet        |  5 KB
21-08-03 12:18:58 | S3\staging\dms\company_rds\customers\customer_details\LOAD00000001.parquet |  5 KB
21-08-03 12:18:58 | S3\staging\dms\company_rds\devices\device_models\LOAD00000001.parquet      |  5 KB
21-08-03 12:18:58 | S3\staging\dms\company_rds\devices\devices\LOAD00000001.parquet            |  5 KB
